# Analysis of the Police Figures and Crime in Scotland



## Dataset
![Scottish Statistics Logo](https://images.squarespace-cdn.com/content/v1/600e7b160eec4a7d0511f762/1620835698639-NAWDXF41R9FBJEZ8VZ97/SG+logo.png)

In this example we will be using a crime dataset from the Scottish Statistics Website located here - [Scottish Government Statistics](https://statistics.gov.scot/home) <br>
The Scottish Crime Dataset contains data spanning from 1996 to 2023. <br>
Other datasets from the Scottish Government will be used including Poverty Data, Scottish Police Numbers <br>

Data can be found directly from the following links:<br>
https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Frecorded-crime <br>
https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpoverty-all-people <br>
https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpolice-officer-quarterly-strength <br>

In [1]:
import pandas as pd
import numpy as np
import os

## Importing data
Data will be directly read as csv from the http source as shown above.

In [2]:
df_crime = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Frecorded-crime')
print('df_crime created')
#df_poverty = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpoverty-all-people')
#print('df_poverty created')
#df_police = pd.read_csv('https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Fpolice-officer-quarterly-strength')
#print('df_police created')

df_crime created


## Data Pre-Processing

### Set Data Types and Rename Columns

In [3]:
df_crime = df_crime.astype({'FeatureCode': 'object', 
                            'FeatureName': 'object', 
                            'FeatureType':'object', 
                            'DateCode':'object',
                            'Measurement':'object',
                            'Units':'object',
                            'Value':'int64',
                            'Crime or Offence':'object'})

df_crime.rename(columns={   'FeatureCode': 'code', 
                            'FeatureName': 'area', 
                            'FeatureType':'area_type', 
                            'DateCode':'date',
                            'Measurement':'measurement',
                            'Units':'units',
                            'Value':'value',
                            'Crime or Offence':'crime_category'},
                             inplace=True)

df_crime.dtypes

code              object
area              object
area_type         object
date              object
measurement       object
units             object
value              int64
crime_category    object
dtype: object

### Pivot Measurement Column

In [4]:
df_crime_pivoted = pd.pivot_table(data = df_crime,
                    index = ['code', 'area', 'area_type', 'date', 'crime_category'],
                    columns = ['measurement'],
                    values = 'value')

df_crime_pivoted.columns.name= None
df_crime_pivoted.reset_index(inplace=True)

df_crime = df_crime_pivoted

df_crime.rename(columns={'Count':'count', 'Ratio':'count_per_10k'}, inplace=True)

In [5]:
df_crime.head()

,code,area,area_type,date,crime_category,count,count_per_10k
0,S12000005,Clackmannanshire,Council Area,1996/1997,All Crimes,2852.0,587.0
1,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 1: Non-sexual crimes of violence,581.0,120.0
2,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 2: Sexual crimes,24.0,5.0
3,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 3: Crimes of dishonesty,1245.0,256.0
4,S12000005,Clackmannanshire,Council Area,1996/1997,All Group 4: Damage and reckless behaviour,588.0,121.0


### Remove Aggregate Crime Category Rows

In [6]:
df_crime = df_crime[~df_crime['crime_category'].str.startswith('All ')]

df_crime = df_crime.sort_values(by=['date','area','count_per_10k', 'count'], 
                                ascending=[True, True, False, False])

### Split Date Column

In [7]:
df_crime[['date_start', 'date_end']] = df_crime['date'].str.split('/', n=1, expand=True)
df_crime.drop(columns=['date'], inplace=True)
df_crime = df_crime.astype({'date_start':'int64', 'date_end':'int64'})

### Remove Scotland Rows

In [8]:
df_crime_scotland = df_crime[df_crime['area'] == 'Scotland']

df_crime.drop(df_crime[df_crime['area'] == 'Scotland'].index, inplace=True)

### Bin Count Categories

In [9]:
count_10k_bins = np.linspace(min(df_crime['count_per_10k']), max(df_crime['count_per_10k']), 8)

count_10k_bins_names = ['Minimal', 'Very Low', 'Low', 'Medium', 'High', 'Very High', 'Extremely High']

df_crime['count_category_historic'] = pd.cut(df_crime['count_per_10k'], 
                                             count_10k_bins, 
                                             labels=count_10k_bins_names, 
                                             include_lowest=True)

In [10]:
mapping_bins_annual = pd.DataFrame(columns=['date_start', 'annual_bins'])
df_crime_concatted = df_crime.copy()

for year in df_crime['date_start'].unique():
    df_temp = df_crime[df_crime['date_start'] == year]
    count_10k_bins = np.linspace(min(df_temp['count_per_10k']), max(df_temp['count_per_10k']), 8)
    data = {'date_start':year, 'annual_bins':[count_10k_bins]}
    df_insert = pd.DataFrame(data)
    mapping_bins_annual = pd.concat([mapping_bins_annual, df_insert])

In [11]:
df_crime = df_crime.reset_index(drop=True)

In [12]:
df_crime.describe(include='all')

,code,area,area_type,crime_category,count,count_per_10k,date_start,date_end,count_category_historic
count,33983,33983,33983,33983,33983.000000,33983.000000,33983.000000,33983.000000,33983
unique,32,32,1,50,NaN,NaN,NaN,NaN,7
top,S12000033,Aberdeen City,Council Area,Crimes: Group 3: Other theft,NaN,NaN,NaN,NaN,Minimal
freq,1062,1062,33983,864,NaN,NaN,NaN,NaN,29595
mean,NaN,NaN,NaN,NaN,459.971191,24.415737,2010.454198,2011.454198,NaN
std,NaN,NaN,NaN,NaN,1178.436115,40.780576,7.714958,7.714958,NaN
min,NaN,NaN,NaN,NaN,0.000000,0.000000,1996.000000,1997.000000,NaN
25%,NaN,NaN,NaN,NaN,21.000000,2.000000,2004.000000,2005.000000,NaN
50%,NaN,NaN,NaN,NaN,91.000000,7.000000,2011.000000,2012.000000,NaN
75%,NaN,NaN,NaN,NaN,378.000000,26.000000,2017.000000,2018.000000,NaN


In [13]:
df_crime['count_category_historic'].value_counts(normalize=True).round(5) * 100

count_category_historic
Minimal           87.088
Very Low           8.775
Low                2.901
Medium             0.859
High               0.282
Very High          0.082
Extremely High     0.012
Name: proportion, dtype: float64

In [20]:
df_crime.sample(20)

,code,area,area_type,crime_category,count,count_per_10k,date_start,date_end,count_category_historic
10683,S12000048,Perth and Kinross,Council Area,Crimes: Group 5: Other crimes against society,3.0,0.0,2006,2007,Minimal
278,S12000008,East Ayrshire,Council Area,Crimes: Group 2: Crimes associated with prosti...,0.0,0.0,1996,1997,Minimal
16419,S12000011,East Renfrewshire,Council Area,Crimes: Group 4: Fire-raising,34.0,4.0,2011,2012,Minimal
11719,S12000038,Renfrewshire,Council Area,Crimes: Group 3: Theft from a motor vehicle,742.0,43.0,2007,2008,Minimal
25189,S12000049,Glasgow City,Council Area,Crimes: Group 5: Drugs - Possession,5435.0,88.0,2017,2018,Very Low
26780,S12000017,Highland,Council Area,Crimes: Group 3: Fraud,260.0,11.0,2018,2019,Minimal
20627,S12000011,East Renfrewshire,Council Area,Crimes: Group 3: Theft from a motor vehicle,25.0,3.0,2014,2015,Minimal
8457,S12000018,Inverclyde,Council Area,Crimes: Group 2: Other sexual crimes,4.0,0.0,2004,2005,Minimal
1388,S12000014,Falkirk,Council Area,Crimes: Group 1: Robbery,34.0,2.0,1997,1998,Minimal
23484,S12000045,East Dunbartonshire,Council Area,Offences: Group 8: Seat belt offences,20.0,2.0,2016,2017,Minimal
